In [ ]:
# audio feature related
import glob
import os
import librosa
import numpy as np
import csv

# lyrical feature related
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import csv
import numpy as np

In [ ]:
# audio feature extraction methods
def load_sound_files(parent_dir, file_paths):
    m_features = ["file","mean_mfccs","mean_chroma","mean_mel","mean_contrast","mean_tonnetz"]
    m_features += ["stddev_mfccs","stddev_chroma","stddev_mel","stddev_contrast","stddev_tonnetz"]
    ofile  = open('data.csv', 'w')
    wr = csv.writer(ofile, quoting=csv.QUOTE_ALL)
    #save headers
    wr.writerow(m_features)
    for fp in file_paths:
        print("file: "+fp+".mp3")
        try:
            #load file
            X,sample_rate = librosa.load(parent_dir+"/"+fp+".mp3")
            #extract feature
            ##Short fourier transform
            stft = np.abs(librosa.stft(X))
            ##Mel-frequency cepstral coefficients
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
            mean_mfccs = np.mean(mfccs)
            stddev_mfccs = np.std(mfccs)
            ##Chromagram of a short-time Fourier transform
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            mean_chroma = np.mean(chroma)
            stddev_chroma = np.std(chroma)
            ##Mel-scaled power spectrogram
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            mean_mel = np.mean(mel)
            stddev_mel = np.std(mel)
            ##Octave-based spectral contrast
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            mean_contrast = np.mean(contrast)
            stddev_contrast = np.std(contrast)
            ##Tonnetz
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            mean_tonnetz = np.mean(tonnetz)
            stddev_tonnetz = np.std(tonnetz)
            #store features in a list
            m_features = [fp]
            m_features += [mean_mfccs,mean_chroma,mean_mel,mean_contrast,mean_tonnetz,
                           stddev_mfccs,stddev_chroma,stddev_mel,stddev_contrast,stddev_tonnetz]
            print(m_features)
            #save list in csv
            wr.writerow(m_features)
        except FileNotFoundError:
            print("The file does not exist")
    ofile.close()
    print("DONE")
    ifile  = open('data.csv', "r")
    reader = csv.reader(ifile)
    for row in reader:
        print(row)
    ifile.close()

def append_sound_files(parent_dir, file_paths):
    ofile  = open('data.csv', 'a')
    wr = csv.writer(ofile, quoting=csv.QUOTE_ALL)
    for fp in file_paths:
        print("file: "+fp+".mp3")
        try:
            #load file
            X,sample_rate = librosa.load(parent_dir+"/"+fp+".mp3")
            #extract feature
            ##Short fourier transform
            stft = np.abs(librosa.stft(X))
            ##Mel-frequency cepstral coefficients
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
            mean_mfccs = np.mean(mfccs)
            stddev_mfccs = np.std(mfccs)
            ##Chromagram of a short-time Fourier transform
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            mean_chroma = np.mean(chroma)
            stddev_chroma = np.std(chroma)
            ##Mel-scaled power spectrogram
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            mean_mel = np.mean(mel)
            stddev_mel = np.std(mel)
            ##Octave-based spectral contrast
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            mean_contrast = np.mean(contrast)
            stddev_contrast = np.std(contrast)
            ##Tonnetz
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            mean_tonnetz = np.mean(tonnetz)
            stddev_tonnetz = np.std(tonnetz)
            #store features in a list
            m_features = [fp]
            m_features += [mean_mfccs,mean_chroma,mean_mel,mean_contrast,mean_tonnetz,
                           stddev_mfccs,stddev_chroma,stddev_mel,stddev_contrast,stddev_tonnetz]
            print(m_features)
            #save list in csv
            wr.writerow(m_features)
        except FileNotFoundError:
            print("The file does not exist")
    ofile.close()
    print("DONE")
    ifile  = open('data.csv', "r")
    reader = csv.reader(ifile)
    for row in reader:
        print(row)
    ifile.close()    

In [ ]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

    
# load all docs in a directory
def process_docs(directory, vocab):
    # walk through all files in the folder
    filenames = listdir(directory)
    filenames.sort()
    print("Total # of files:",len(filenames))
    for filename in filenames:
        # skip any reviews in the test set
#         if filename.startswith('cv9'):
#             continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab)


# save list to file
def save_list(lines, filename):
        # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()


# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)


# load all docs in a directory and place data in a list
def read_docs(directory, vocab):
    lines = list()
    filenames = listdir(directory)
    filenames.sort()
    # walk through all files in the folder
    for filename in filenames:
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        line = doc_to_line(path, vocab)
        # add to list
        lines.append(line)
    return lines

In [ ]:
audio_valence = []
audio_arousal = []
lyrical_valence = []
lyrical_arousal = []
exoected_valence = []
expected_arousal = []
# average 
predicted_valence = (audio_valence + lyrical_valence) / 2
